# Query and download HST data

In this notebook, we will learn how to use `astroquery` and `rocky-worlds-utils` to query and download STIS data related to the RWDDT program.

- Author: Leonardo Dos Santos (STScI)
- Date first published: November 19, 2025

## Imports

- `astropy`: FITS files I/O and units
- `astroquery`: query MAST data
- `rocky_worlds_utils`: various functions to facilitate data analysis
- `os`, `glob`, `shutil`, `pathlib`: tools for managing files and folder

In [ ]:
from astropy.io import fits
import astropy.units as u
from astroquery.mast import Mast
from astroquery.mast import Observations
import glob
import os
from pathlib import Path
from rocky_worlds_utils.hst import tools
import shutil

The following cell contains a function that will be useful for moving all the data from the nested folders, which is the default of `astroquery.mast`, to a single folder.

In [ ]:
def consolidate_files(data_path):
    '''
    Consolidate all files to single directory; necessary for HASP script run.
    ---------------
    Input:
    str data_path : ./mastDownload/HST folders paths; files to be moved here
    ---------------
    Output:
    None. Files moved to data_path. ./mastDownload/HST directory is deleted.
    '''
    # The path to all obs_id folders
    mast_path = f"{data_path}/mastDownload/HST/"

    try:
        # Check if mastDownload exists
        if not os.path.exists(mast_path):
            print(f"Directory {mast_path} doesn't exist.")
            return

        # Get a list of the obs_id paths in mastDownload
        obs_id_dirs = os.listdir(mast_path)

        # Iterate through each obs_id folder and move the files
        for obs_id in obs_id_dirs:
            obs_id_path = os.path.join(mast_path, obs_id)
            files = glob.glob(obs_id_path + "/*fits")

            for file in files:
                file_path = Path(file)
                new_path = data_path / file_path.name
                shutil.move(file, new_path)

        # Now we can remove the mastDownload directory
        if os.path.exists(mast_path):
            shutil.rmtree(f"{data_path}/mastDownload")

    except Exception as e:
        print(f"An error occurred: {e}")

The first step is to query the MAST archive for our observations. In this example, we are interested in all observations of GJ 3929 obtained with the STIS spectrograph and the G140M grating.

In [ ]:
target_name = 'GJ 3929'
search_radius = 10 * u.arcsec
data_query = Observations.query_criteria(objectname=target_name,
                                       radius=search_radius, 
                                       filters="G140M",
                                       instrument_name='STIS/FUV-MAMA')

The following cell retrieves a list of all MAST products related to the query above. We won't need all of these, so we download only those relevant for our data analysis. In this case, that is all the CALSTIS products. We will download them to a specific folder.

In [ ]:
mast_products = Observations.get_product_list(
    data_query
)

data_folder = "./example_data/"

_ = Observations.download_products(
    mast_products,
    download_dir=data_folder,
    project='CALSTIS'
)

Lastly, we consolidate all downloaded files to the `data_folder`.

In [ ]:
consolidate_files(Path(data_folder))

COS data can be queried and downloaded in a similar manner, simply by modifying `Observations.query_criteria()` and selecting CALCOS products when using `Observations.download_products()`.